In [2]:
import pandas as pd
import numpy as np
from sklearn import tree
from collections import Counter
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from tabulate import tabulate

In [3]:
df = pd.read_csv("new_train.csv")

In [6]:
df['rank_rel'] = np.zeros(len(df))
df['rank_rel'] = np.where(df.click_bool == 1, 1, df.rank_rel)
df['rank_rel'] = np.where(df.booking_bool == 1, 5, df.rank_rel)

In [5]:
df= df.drop(['Unnamed: 0'], axis = 1)

In [7]:
sample_srch_id = np.random.choice(df.srch_id.unique(), size= int(0.1 * len(df.srch_id.unique())))
df_sample = df[df.srch_id.isin(sample_srch_id)]
print len(df_sample)
print df_sample['click_bool'].corr(df['click_bool'])
print df_sample['booking_bool'].corr(df['booking_bool'])

471048
0.9999999999999999
0.9999999999999998


In [8]:
grouped  = df_sample[df_sample.click_bool == False].groupby('srch_id')
grouped = grouped.apply(lambda x: x.sample(frac = 0.2))
grouped = grouped.apply(lambda x: x) 
df_sample = df_sample.drop(df_sample[df_sample.click_bool == False].index).append(grouped)

In [18]:
y_rank = df_sample[['rank_rel']]
X = df_sample.drop(['position','rank_rel','click_bool','booking_bool'], axis = 1)

In [20]:
X_train = X.head(2*len(df)/3) 
X_test = X.tail(len(df)/3) 
y_train = y_rank.head(2*len(df)/3) 
y_test = y_rank.tail(len(df)) 

In [22]:
gbm = GradientBoostingClassifier()
model = gbm.fit(X_train, y_train)

C:\Users\Maria\Anaconda2\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [23]:
pred2 = model.predict(X_test)

In [24]:
print gbm.score(X_test, y_test)

0.8146399912857195


In [26]:
print sum(pred2)

10166.0


In [27]:
gbm = GradientBoostingClassifier(max_depth = 2)

In [28]:
y_test2 = df_sample[['position']].loc[X_test.index]
y_train2 = df_sample[['position']].loc[X_train.index]

In [ ]:
model2 = gbm.fit(X_train, y_train2)

In [ ]:
pred = model2.predict(X_test)

In [ ]:
print gbm.score(X_test, y_test2)

In [ ]:
print pred

In [ ]:
dfFinal = X_test

In [ ]:
dfFinal['position'] = pred2
df['rank_rel'] = pred

In [ ]:
dfFinal = dfFinal.sort_values(by=['srch_id'])

In [ ]:
df = df[['srch_id', 'position', 'rank_rel']]



def dcg(i, rel):

    return (2**rel - 1)/(np.log2(i + 1))



#########################################################################################



# CACULATING DCG



sum_dcg = 0

array_dcg = []



for k in range(0, len(df) - 1):

    if k != len(df) - 1:
        
        if(df['srch_id'].iloc[k] < df['srch_id'].iloc[k + 1]):
 

            sum_dcg += dcg(df['position'].iloc[k],df['rank_rel'].iloc[k])

            array_dcg.append(sum_dcg)
            sum_dcg = 0

        else:

            sum_dcg += dcg(df['position'].iloc[k],df['rank_rel'].iloc[k])

    else:

        sum_dcg += dcg(df['position'].iloc[k],df['rank_rel'].iloc[k])

        array_dcg.append(sum_dcg)

        sum_dcg = 0

# CALCULATING IDCG
array_idcg = np.array([])
t_idcg = np.array([])
sum_idcg = 0

for k in range(0, len(df) - 1):
    if k != len(df) - 1:
        if(df['srch_id'].iloc[k] < df['srch_id'].iloc[k + 1]):
            t_idcg = np.append(t_idcg, df['rank_rel'].iloc[k])
            t_idcg = np.sort(t_idcg)[::-1]
            for j in range(0, len(t_idcg)):
                sum_idcg += dcg(j + 1, t_idcg[j])
                if(sum_idcg > 0):
                    print sum_idcg
            array_idcg = np.append(array_idcg, sum_idcg)
            sum_idcg = 0
            t_idcg = []
        else:
            t_idcg = np.append(t_idcg, df['rank_rel'].iloc[k])   
    else:
        t_idcg = np.append(t_idcg, df['rank_rel'].iloc[k])
        t_idcg = np.sort(t_idcg)[::-1]
        for j in range(0, len(t_idcg)):
            sum_idcg += dcg(j + 1, t_idcg[j])
        array_idcg = np.append(array_idcg, sum_idcg)
        
print 'nDGC = ', (np.sum(array_dcg)/np.sum(array_idcg))
